In [21]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from urllib.parse import urljoin
import re
import os
from bs4 import BeautifulSoup

In [14]:
BASE = "https://api.cer.gov.au/datahub-public/v1/"
SCHEME_ID = "NGER"
YEAR_MIN, YEAR_MAX = 2014, 2024
SAVE_DIR = "./nger_csv"
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/124.0.0.0 Safari/537.36"
})


In [5]:
catalog_url = urljoin(BASE, f"api/Schemes/{SCHEME_ID}/DatasetCatalogItems")
def get_json(url, params=None, timeout=15):
    r = session.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    return r.json()
catalog = get_json(catalog_url)
catelog_items = catalog.get("items", catalog) if isinstance(catalog, dict) else catalog

In [18]:
def download_file(url,filepath,timeout=30):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1<<14):
                if chunk: f.write(chunk)

In [8]:
pattern = re.compile(
r"Greenhouse and energy information by designated generation facility\s*(20\d{2})[–-](\d{2})",    re.IGNORECASE
)

In [20]:
dataset_url =urljoin(BASE,f'api/Dataset/{SCHEME_ID}/dataset/')
for item in catelog_items:
    title = item.get("displayName")
    m = pattern.search(title)
    if m:
        start_year = int(m.group(1))
        end_suffix = int(m.group(2))
        end_year = int(str(start_year)[:2] + f"{end_suffix:02d}")
        if start_year < YEAR_MIN or end_year > YEAR_MAX:
            continue
        dataset_id = item.get("id")
        download_url = urljoin(dataset_url, f'{dataset_id}.csv')
        print(f"{download_url}")
        save_path = os.path.join(SAVE_DIR, f"{start_year}-{end_year}.csv")
        print(f"{title}  -> {start_year}-{end_year}, datasetId={dataset_id}")
        try:
            print(f'Downloading -> {save_path}')
            download_file(download_url, save_path)
            time.sleep(1)
        except requests.HTTPError as e:
            print(f"HTTP Error {e.response.status_code}")
print("All files downloaded")

https://api.cer.gov.au/datahub-public/v1/api/Dataset/NGER/dataset/ID0083.csv
Greenhouse and energy information by designated generation facility 2022-23  -> 2022-2023, datasetId=ID0083
https://api.cer.gov.au/datahub-public/v1/api/Dataset/NGER/dataset/ID0079.csv
Greenhouse and energy information by designated generation facility 2018-19  -> 2018-2019, datasetId=ID0079
https://api.cer.gov.au/datahub-public/v1/api/Dataset/NGER/dataset/ID0082.csv
Greenhouse and energy information by designated generation facility 2021-22  -> 2021-2022, datasetId=ID0082
https://api.cer.gov.au/datahub-public/v1/api/Dataset/NGER/dataset/ID0243.csv
Greenhouse and energy information by designated generation facility 2023–24  -> 2023-2024, datasetId=ID0243
https://api.cer.gov.au/datahub-public/v1/api/Dataset/NGER/dataset/ID0080.csv
Greenhouse and energy information by designated generation facility 2019-20  -> 2019-2020, datasetId=ID0080
https://api.cer.gov.au/datahub-public/v1/api/Dataset/NGER/dataset/ID0076.cs

### ABS Download Files

In [39]:
PAGE_URL = 'https://cer.gov.au/markets/reports-and-data/large-scale-renewable-energy-data'
abs_out_path = "./abs_csv"
response = requests.get(PAGE_URL,timeout=15)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")
#all the files download url are the brother node under the h4 title, but after first try the h4 only has text <h4> Raw data in CSV format</h4> it has no id in the response, but at the website it  has an id called raw-data-in-csv-format so it has to be changed to find all h4 then do the regex
for h4 in soup.find_all("h4"):
    if "raw data in csv format" in h4.get_text(strip=True).lower():
        #the link is in the first brother node under the h4 which is <a> and the link is in the parameter href
        a =h4.find_next("a",attrs={"download":True})
        #it is half of the download url it needs to concat with the base url, the full link can be tested and which is just the page url + href
        href = a["href"]
        csv_url = urljoin(PAGE_URL,href)
        if "power-stations-and-projects" in href or "power stations and projects" in href.lower():
            print(f"Downloading {csv_url}")
            file_name = os.path.basename(href)+".csv" 
            download_file(csv_url,os.path.join(abs_out_path,file_name))
            time.sleep(1)
            print(f"Downloaded {file_name}")
        print("--------------------------------------------------------")
print("All files downloaded")

--------------------------------------------------------
Downloaded power-stations-and-projects-accredited.csv
--------------------------------------------------------
Downloaded power-stations-and-projects-committed.csv
--------------------------------------------------------
Downloaded power-stations-and-projects-probable.csv
--------------------------------------------------------
--------------------------------------------------------
All files downloaded


### ABS Data by Regions

In [43]:
ABS_REGION_URL = "https://www.abs.gov.au/methodologies/data-region-methodology/2011-24#data-downloads"
abs_region_path = "./abs_region_xslx"
def download_file_from_abs_by_region(ABS_REGION_URL,region,filepath,timeout=30):
    response = requests.get(ABS_REGION_URL,timeout=15)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    h4 = soup.find("h4",string=lambda t: t and region in t)
    if not h4:
        raise SystemExit("Not find such h4")
    parent = h4.find_parent("l1",class_="field__item")
    a = parent.find_next("a",href=True) if parent else h4.find_next("a",href=True)
    if not a:
        raise SystemExit("Not find such download link")
    href = a["href"]
    file_url = urljoin(ABS_REGION_URL,href)
    print(f"Downloading {file_url}")
    label = a['aria-label']
    file_name =label+".xlsx"
    download_file(file_url,os.path.join(filepath,file_name))
    print(f"Downloaded {file_name}")

In [44]:
economy_region='Economy and industry, ASGS and LGA, 2011, 2016-2024'
population_region ="Population and people, ASGS, LGA, and RA, 2011, 2016-2024"
download_file_from_abs_by_region(ABS_REGION_URL,economy_region,abs_region_path)
download_file_from_abs_by_region(ABS_REGION_URL,population_region,abs_region_path)

Downloaded Download - Economy and industry, ASGS and LGA, 2011, 2016-2024 xlsx 18.75 MB.xlsx
Downloaded Download - Population and people, ASGS, LGA, and RA, 2011, 2016-2024 xlsx 26.03 MB.xlsx
